In [10]:
from optimization.acs import ACS_VRP,ACS_TSP
from optimization.bso import BSO_VRP,BSO_TSP
from optimization.ga import GA_VRP
from optimization.sa import SA_VRP
from optimization.tabu_search import TS_VRP
from optimization.dka import DKA_VRP
from optimization.aco import ACO_VRP,ACO_TSP
from optimization.haco import HACO_VRP,HACO_TSP
from optimization.kma import KMA_VRP
from optimization.woa import WOA_VRP
from optimization.vns import VNS_VRP
from optimization.pso import PSO_VRP

from optimization.koneksi import ConDB

import sql_connection
import json
import time
import random
import datetime
import copy
import pandas as pd
import numpy as np
import scipy.stats as stats

import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
from matplotlib.patches import FancyArrowPatch

import warnings
warnings.filterwarnings('ignore')

# Generate Data

In [2]:
def generate_data(n = 30, random_state = None):
    random.seed(random_state)
    query = """SELECT 
                    p.post_id,
                    p.post_type,
                    CASE
                        WHEN pj.pj_jam_buka = pj.pj_jam_tutup THEN "tutup"
                        ELSE "buka"
                    END AS is_operate
                FROM 
                    posts p
                LEFT JOIN
                    posts_jadwal pj
                    ON p.post_id = pj.pj_id_tempat AND pj.pj_hari = "minggu"
                """

    df_location = sql_connection.read_from_sql(query)
    
    tourid = df_location[(df_location['post_type']=="location")&
                         (df_location['is_operate'] != "tutup")]['post_id'].values.tolist()
    tourid = random.sample(tourid,n)
    idhotel = df_location[df_location['post_type']=="hotel"]['post_id'].values.tolist()
    idhotel = idhotel[random.randint(0,len(idhotel)-1)]
    
    db = ConDB()

    hotel = db.HotelbyID(idhotel)
    tur = db.WisatabyID(tourid)
    timematrix = db.TimeMatrixbyID(hotel._id,tourid)
    
    return hotel,tur,timematrix#,tourid,idhotel

# Analisis


In [3]:
def find_node(node_id,tour):
    for node in tour:
        if node._id == node_id:
            return node
    return False

def time_to_second(time):
    return (time.hour*3600)+(time.minute*60)+time.second

def diff_second_between_time(time_a,time_b):
    #input: time_a and time_b, datetime.time()
    #output: time_b - time_a, seconds (int)
    return time_to_second(time_b) - time_to_second(time_a)

def get_attribute_analysis(solution):
    days_needed = len(solution)
    poi_included = len(sum([sol["index"] for sol in solution],[]))
    avg_rating = sum(sum([sol["rating"] for sol in solution],[]))/poi_included
    total_tarif = sum(sum([sol["tarif"] for sol in solution],[]))
    total_duration = sum([diff_second_between_time(j[0],j[-1]) for j in [i['waktu'] for i in solution]])
    return days_needed,poi_included,avg_rating,total_tarif,total_duration

def get_analysis(model,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating):
    start = time.time()
    model.set_model(tour=tur,hotel=hotel,timematrix=timematrix,
                    travel_days=travel_days,
                    degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    solution,fitness = model.construct_solution()[-2:]
    end = time.time()
    run_time = end-start
    
    daily_fitness = []
    for day in solution:
        daily_fitness.append(model.MAUT([day],use_penalty=False))
    days_needed,poi_included,avg_rating,total_tarif,total_duration = get_attribute_analysis(solution)
    results = {"solution":[sol["index"] for sol in solution],
               "fitness":fitness,
               "daily_fitness":daily_fitness,
               "days_needed":days_needed,
               "poi_included":poi_included,
               "avg_rating":avg_rating,
               "total_tarif":total_tarif,
               "total_duration":total_duration,
               "run_time":run_time}
    
    return results

In [4]:
def get_detail_node_on_solution(solution):
    db = ConDB()
    node_solution = []
    for i in solution:
        nodes = db.WisatabyID(i)
        node_solution.append(nodes)
    return node_solution

def visualize_solution(solution_node,hotel):
    # Sample VRP data
    vrp_data = [[hotel]+sol+[hotel] for sol in solution_node]
    
    # Generate random node coordinates
    node_coords = {node._id: (float(node.long), float(node.lat)) for node in set(node for route in vrp_data for node in route)}  

    # Number of routes (you can calculate this)
    num_routes = len(vrp_data)

    # Create the plot
    fig, ax = plt.subplots(figsize=(12,9))

    # Define the colors for routes using a colormap
    cmap = plt.get_cmap('tab20', num_routes)

    for i, route in enumerate(vrp_data):
        x = [node_coords[node._id][0] for node in route]
        y = [node_coords[node._id][1] for node in route]
        color = cmap(i)  # Get the color for the route
        # Plot the nodes (scatter plot)
        ax.scatter(x, y, c=color, label=f'Route {i + 1}', s=100)

        for j in range(len(x) - 1):
            # Make the arrow heads larger by increasing mutation_scale
            arrow = FancyArrowPatch(
                (x[j], y[j]), (x[j + 1], y[j + 1]),
                arrowstyle='->', mutation_scale=15, color=color
            )
            ax.add_patch(arrow)

    # Set node 159 (start and end point) to black
    start_x, start_y = node_coords[hotel._id]
    ax.scatter(start_x, start_y, c='k', label='Hotel', s=100)

    # Add labels for nodes
    for node, (x, y) in node_coords.items():
        ax.annotate(str(node), (x, y), textcoords="offset points", xytext=(0, 10), ha='center')

    # Add legend
    ax.legend()

    plt.show()



## Outlier Sensitivity

In [6]:
hotel,tur,timematrix = generate_data(n=87,random_state=40)
dwaktu,dtarif,drating = 1,0,0

In [5]:
id_outlier = [39, 58, 62, 64, 65]

In [7]:
min_days = 1
max_days = 13

In [11]:
outlier_sensitivity_analysis = {1 : {},
                                2 : {},
                                3 : {},
                                4 : {},
                                5 : {},
                                6 : {},
                                7 : {},
                                8 : {},
                                9 : {},
                                10 : {},
                                11 : {},
                                12 : {},
                               13 : {}}

for n_days in outlier_sensitivity_analysis:
    travel_days = n_days
    
    pso_vrp = PSO_VRP(random_state=100)
    results_pso_vrp = get_analysis(pso_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    num_outlier = len([outlier for outlier in id_outlier if outlier in sum(results_pso_vrp['solution'],[])])
    outlier_sensitivity_analysis[travel_days]["pso_vrp"] = num_outlier

In [12]:
outlier_sensitivity_analysis

{1: {'pso_vrp': 0},
 2: {'pso_vrp': 0},
 3: {'pso_vrp': 0},
 4: {'pso_vrp': 1},
 5: {'pso_vrp': 2},
 6: {'pso_vrp': 2},
 7: {'pso_vrp': 2},
 8: {'pso_vrp': 1},
 9: {'pso_vrp': 0},
 10: {'pso_vrp': 3},
 11: {'pso_vrp': 3},
 12: {'pso_vrp': 3},
 13: {'pso_vrp': 4}}

In [13]:
result = pd.DataFrame()
for i in outlier_sensitivity_analysis:
    if i not in range(min_days,max_days+1):
        continue
    else:
        travel_days = i
        methods = []
        num_outlier = []
        for j in outlier_sensitivity_analysis[i]:
            methods.append(j)
            num_outlier.append(outlier_sensitivity_analysis[i][j])
        temp_res = pd.DataFrame({'methods':methods,
                                'num_outlier':num_outlier})
        temp_res['travel_days'] = travel_days
        result = pd.concat([result,temp_res])

In [14]:
result

,methods,num_outlier,travel_days
0,pso_vrp,0,1
0,pso_vrp,0,2
0,pso_vrp,0,3
0,pso_vrp,1,4
0,pso_vrp,2,5
0,pso_vrp,2,6
0,pso_vrp,2,7
0,pso_vrp,1,8
0,pso_vrp,0,9
0,pso_vrp,3,10


In [15]:
result.to_excel('pso_outlier_sensitivity.xlsx',index=False)

## General Performance (50 random nodes)

In [16]:
def result_to_xlsx_rd_nodes(result,random_ls,filename):
    df = pd.DataFrame()
    for method in result:
        res_method = result[method]
        df_method = {
            "random_seed" : random_ls,
            "fitness" : [],
            "fitness_1" : [],
            "fitness_2" : [],
            "fitness_3" : [],
            "poi_included" : [],
            "rating" : [],
            "tarif" : [],
            "total_duration" : [],
            "run_time" : [],
        }
        for res in res_method:
            df_method["fitness"].append(res['fitness'])
            df_method["fitness_1"].append(res['daily_fitness'][0])
            df_method["fitness_2"].append(res['daily_fitness'][1])
            df_method["fitness_3"].append(res['daily_fitness'][2])
            df_method["poi_included"].append(res['poi_included'])
            df_method["rating"].append(res['avg_rating'])
            df_method["tarif"].append(res['total_tarif'])
            df_method["total_duration"].append(res['total_duration'])
            df_method["run_time"].append(res['run_time'])
        df_method = pd.DataFrame(df_method)
        df_method['method'] = method
        df = pd.concat([df,df_method])
    df.to_excel(filename,index=False)

### All DOI 1

In [17]:
random_ls = list(range(5,(5*50)+1,5))
random_node_analysis ={
    "pso_vrp": []
}
for i in range(len(random_ls)):
    # setting
    hotel,tur,timematrix = generate_data(n=30,random_state=random_ls[i]) #random state 50 blacklist
    travel_days = 3
    dwaktu,dtarif,drating = 1,1,1
    
    print("random state = ",random_ls[i])
    
    pso_vrp = PSO_VRP(random_state=100)
    results_pso_vrp = get_analysis(pso_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["pso_vrp"].append(results_pso_vrp)

random state =  5
random state =  10
random state =  15
random state =  20
random state =  25
random state =  30
random state =  35
random state =  40
random state =  45
random state =  50
random state =  55
random state =  60
random state =  65
random state =  70
random state =  75
random state =  80
random state =  85
random state =  90
random state =  95
random state =  100
random state =  105
random state =  110
random state =  115
random state =  120
random state =  125
random state =  130
random state =  135
random state =  140
random state =  145
random state =  150
random state =  155
random state =  160
random state =  165
random state =  170
random state =  175
random state =  180
random state =  185
random state =  190
random state =  195
random state =  200
random state =  205
random state =  210
random state =  215
random state =  220
random state =  225
random state =  230
random state =  235
random state =  240
random state =  245
random state =  250


In [18]:
random_node_analysis

{'pso_vrp': [{'solution': [[88, 55, 51, 14, 35, 7, 74],
    [52, 4, 90, 26, 50, 10, 66, 22],
    [65, 99, 34, 15, 2, 9]],
   'fitness': 0.6755950301154705,
   'daily_fitness': [0.6662901192971948,
    0.6765854297693918,
    0.6362246680642907],
   'days_needed': 3,
   'poi_included': 21,
   'avg_rating': 4.604761904761905,
   'total_tarif': 44500,
   'total_duration': 124620,
   'run_time': 0.567946195602417},
  {'solution': [[46, 71, 99, 51, 57],
    [44, 39, 74, 66, 10, 22, 21, 36],
    [6, 18, 12, 49, 32, 2, 89]],
   'fitness': 0.6385194744892385,
   'daily_fitness': [0.585505162241888, 0.631611561783022, 0.6109601067565669],
   'days_needed': 3,
   'poi_included': 20,
   'avg_rating': 4.575,
   'total_tarif': 34500,
   'total_duration': 123780,
   'run_time': 0.7624168395996094},
  {'solution': [[3, 15, 85, 20, 42, 2, 5, 50, 46],
    [68, 39, 53, 93, 43],
    [81, 99, 8, 22, 74, 57, 32, 36]],
   'fitness': 0.5898684298110585,
   'daily_fitness': [0.5514604306024683,
    0.56449654

In [19]:
result_to_xlsx_rd_nodes(random_node_analysis,random_ls,"pso_random_nodes_alldoi_1.xlsx")

### Travel Duration 1, other 0

In [20]:
random_ls = list(range(5,(5*50)+1,5))
random_node_analysis ={
    "pso_vrp": []
}
for i in range(len(random_ls)):
    # setting
    hotel,tur,timematrix = generate_data(n=30,random_state=random_ls[i])
    travel_days = 3
    dwaktu,dtarif,drating = 1,0,0
    
    print("random state = ",random_ls[i])
    
    pso_vrp = PSO_VRP(random_state=100)
    results_pso_vrp = get_analysis(pso_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["pso_vrp"].append(results_pso_vrp)

random state =  5
random state =  10
random state =  15
random state =  20
random state =  25
random state =  30
random state =  35
random state =  40
random state =  45
random state =  50
random state =  55
random state =  60
random state =  65
random state =  70
random state =  75
random state =  80
random state =  85
random state =  90
random state =  95
random state =  100
random state =  105
random state =  110
random state =  115
random state =  120
random state =  125
random state =  130
random state =  135
random state =  140
random state =  145
random state =  150
random state =  155
random state =  160
random state =  165
random state =  170
random state =  175
random state =  180
random state =  185
random state =  190
random state =  195
random state =  200
random state =  205
random state =  210
random state =  215
random state =  220
random state =  225
random state =  230
random state =  235
random state =  240
random state =  245
random state =  250


In [21]:
random_node_analysis

{'pso_vrp': [{'solution': [[4, 14, 10, 90, 35, 88, 15, 50, 2, 7, 52],
    [55, 51, 38, 34, 83, 66, 9],
    [72, 99, 48, 74, 26, 22]],
   'fitness': 0.6611111111111111,
   'daily_fitness': [0.518287037037037,
    0.4537037037037037,
    0.4502314814814815],
   'days_needed': 3,
   'poi_included': 24,
   'avg_rating': 4.575,
   'total_tarif': 167500,
   'total_duration': 123840,
   'run_time': 0.4087867736816406},
  {'solution': [[2, 32, 18, 38, 51, 74, 66],
    [6, 22, 12, 57, 67, 46, 49, 21],
    [44, 48, 99, 10, 80, 5, 36, 89]],
   'fitness': 0.637770061728395,
   'daily_fitness': [0.4467592592592593,
    0.467824074074074,
    0.47870370370370374],
   'days_needed': 3,
   'poi_included': 23,
   'avg_rating': 4.539130434782609,
   'total_tarif': 151500,
   'total_duration': 127020,
   'run_time': 0.3816800117492676},
  {'solution': [[57, 50, 93, 33, 2, 5, 32, 3],
    [81, 48, 71, 85, 22, 15, 53, 36],
    [29, 99, 68, 20, 74]],
   'fitness': 0.6116898148148148,
   'daily_fitness': [0.4

In [22]:
result_to_xlsx_rd_nodes(random_node_analysis,random_ls,"pso_random_nodes_dwaktu_1.xlsx")

### Tarif 1, other 0

In [23]:
random_ls = list(range(5,(5*50)+1,5))
random_node_analysis ={
    "pso_vrp": [],
}
for i in range(len(random_ls)):
    # setting
    hotel,tur,timematrix = generate_data(n=30,random_state=random_ls[i]) #random state 50 blacklist
    travel_days = 3
    dwaktu,dtarif,drating = 0,1,0
    
    print("random state = ",random_ls[i])
    
    pso_vrp = PSO_VRP(random_state=100)
    results_pso_vrp = get_analysis(pso_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["pso_vrp"].append(results_pso_vrp)

random state =  5
random state =  10
random state =  15
random state =  20
random state =  25
random state =  30
random state =  35
random state =  40
random state =  45
random state =  50
random state =  55
random state =  60
random state =  65
random state =  70
random state =  75
random state =  80
random state =  85
random state =  90
random state =  95
random state =  100
random state =  105
random state =  110
random state =  115
random state =  120
random state =  125
random state =  130
random state =  135
random state =  140
random state =  145
random state =  150
random state =  155
random state =  160
random state =  165
random state =  170
random state =  175
random state =  180
random state =  185
random state =  190
random state =  195
random state =  200
random state =  205
random state =  210
random state =  215
random state =  220
random state =  225
random state =  230
random state =  235
random state =  240
random state =  245
random state =  250


In [24]:
random_node_analysis

{'pso_vrp': [{'solution': [[88, 4, 7, 10, 50, 90, 52, 35, 51],
    [55, 14, 2, 99, 65, 15, 9],
    [38, 34, 74, 66, 26, 22]],
   'fitness': 0.8153636734218496,
   'daily_fitness': [0.6132075471698113,
    0.569496855345912,
    0.5849056603773585],
   'days_needed': 3,
   'poi_included': 22,
   'avg_rating': 4.595454545454546,
   'total_tarif': 52500,
   'total_duration': 130500,
   'run_time': 0.6798772811889648},
  {'solution': [[51, 44, 71, 46, 66, 36],
    [39, 49, 99, 22, 57, 89, 10],
    [18, 2, 12, 32, 34, 74, 21]],
   'fitness': 0.8054572271386431,
   'daily_fitness': [0.5778761061946903,
    0.5883480825958702,
    0.6113569321533924],
   'days_needed': 3,
   'poi_included': 20,
   'avg_rating': 4.569999999999999,
   'total_tarif': 31500,
   'total_duration': 129180,
   'run_time': 0.4799330234527588},
  {'solution': [[74, 99, 93, 42, 71],
    [38, 46, 32, 33, 3, 50, 57],
    [64, 81, 2, 15, 85, 22, 36, 43]],
   'fitness': 0.7669216504404266,
   'daily_fitness': [0.53117756142

In [25]:
result_to_xlsx_rd_nodes(random_node_analysis,random_ls,"pso_random_nodes_dtarif_1.xlsx")

### Rating 1, other 0

In [26]:
random_ls = list(range(5,(5*50)+1,5))
random_node_analysis ={
    "pso_vrp": [],
}
for i in range(len(random_ls)):
    # setting
    hotel,tur,timematrix = generate_data(n=30,random_state=random_ls[i]) #random state 50 blacklist
    travel_days = 3
    dwaktu,dtarif,drating = 0,0,1
    
    print("random state = ",random_ls[i])
    
    pso_vrp = PSO_VRP(random_state=100)
    results_pso_vrp = get_analysis(pso_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["pso_vrp"].append(results_pso_vrp)

random state =  5
random state =  10
random state =  15
random state =  20
random state =  25
random state =  30
random state =  35
random state =  40
random state =  45
random state =  50
random state =  55
random state =  60
random state =  65
random state =  70
random state =  75
random state =  80
random state =  85
random state =  90
random state =  95
random state =  100
random state =  105
random state =  110
random state =  115
random state =  120
random state =  125
random state =  130
random state =  135
random state =  140
random state =  145
random state =  150
random state =  155
random state =  160
random state =  165
random state =  170
random state =  175
random state =  180
random state =  185
random state =  190
random state =  195
random state =  200
random state =  205
random state =  210
random state =  215
random state =  220
random state =  225
random state =  230
random state =  235
random state =  240
random state =  245
random state =  250


In [27]:
random_node_analysis

{'pso_vrp': [{'solution': [[4, 48, 15, 88, 7, 14, 2, 10, 50, 9, 35],
    [34, 55, 38, 26, 66, 52],
    [72, 99, 90, 22, 51, 74]],
   'fitness': 0.8241545893719808,
   'daily_fitness': [0.5974747474747476,
    0.46111111111111114,
    0.44259259259259265],
   'days_needed': 3,
   'poi_included': 23,
   'avg_rating': 4.586956521739131,
   'total_tarif': 147500,
   'total_duration': 127560,
   'run_time': 0.5835139751434326},
  {'solution': [[2, 32, 18, 22, 51, 74, 66],
    [6, 46, 38, 10, 48, 57, 21],
    [34, 12, 44, 67, 49, 5, 89, 36]],
   'fitness': 0.7564393939393941,
   'daily_fitness': [0.4023809523809523,
    0.41666666666666696,
    0.38958333333333295],
   'days_needed': 3,
   'poi_included': 22,
   'avg_rating': 4.5590909090909095,
   'total_tarif': 116500,
   'total_duration': 127440,
   'run_time': 0.493333101272583},
  {'solution': [[38, 29, 22, 74, 2, 3, 85],
    [64, 48, 46, 81, 20, 53, 36],
    [33, 68, 32, 47, 57, 5, 50, 15]],
   'fitness': 0.7237323633156966,
   'daily_

In [28]:
result_to_xlsx_rd_nodes(random_node_analysis,random_ls,"pso_random_nodes_drating_1.xlsx")

## Random DOI Combinations

In [29]:
def result_to_xlsx_rd_doi(result,random_doi,random_ls,filename):
    df_doi = pd.DataFrame()
    for doi_res in range(len(result)):
        df = pd.DataFrame()
        for method in result[doi_res]:
            res_method = result[doi_res][method]
            df_method = {
                "random_seed" : random_ls,
                "fitness" : [],
                "fitness_1" : [],
                "fitness_2" : [],
                "fitness_3" : [],
                "poi_included" : [],
                "rating" : [],
                "tarif" : [],
                "total_duration" : [],
                "run_time" : [],
            }
            for res in res_method:
                df_method["fitness"].append(res['fitness'])
                df_method["fitness_1"].append(res['daily_fitness'][0])
                df_method["fitness_2"].append(res['daily_fitness'][1])
                df_method["fitness_3"].append(res['daily_fitness'][2])
                df_method["poi_included"].append(res['poi_included'])
                df_method["rating"].append(res['avg_rating'])
                df_method["tarif"].append(res['total_tarif'])
                df_method["total_duration"].append(res['total_duration'])
                df_method["run_time"].append(res['run_time'])
            df_method = pd.DataFrame(df_method)
            df_method['method'] = method
            df = pd.concat([df,df_method])
        
        doi = random_doi[doi_res]
        df['dwaktu'] = doi[0]
        df['dtarif'] = doi[1]
        df['drating'] = doi[2]
        df = df[['dwaktu','dtarif','drating','method','random_seed','fitness','fitness_1','fitness_2','fitness_3',
                'poi_included','rating','tarif','total_duration','run_time']]
        df_doi = pd.concat([df_doi,df])
#     return df_doi
    df_doi.to_excel(filename,index=False)

In [30]:
random.seed(100)
doi_combi = [(np.round(random.uniform(0,1),2),np.round(random.uniform(0,1),2),np.round(random.uniform(0,1),2)) for i in range(10)]

In [31]:
doi_combi

[(0.15, 0.45, 0.77),
 (0.71, 0.73, 0.43),
 (0.8, 0.53, 0.08),
 (0.46, 0.05, 0.93),
 (0.95, 0.34, 0.31),
 (0.77, 0.2, 0.18),
 (0.19, 0.35, 0.63),
 (0.96, 0.21, 0.96),
 (0.56, 0.9, 0.82),
 (0.16, 0.65, 0.12)]

In [32]:
doi_combi_analysis = []
for doi in doi_combi:
    print("doi combination = ",doi)
    random_ls = list(range(5,(5*50)+1,5))
    random_node_analysis ={
        "pso_vrp": []
    }
    for i in range(len(random_ls)):
        # setting
        hotel,tur,timematrix = generate_data(n=30,random_state=random_ls[i]) #random state 50 blacklist
        travel_days = 3
        dwaktu,dtarif,drating = doi

        print("random state = ",random_ls[i])

        pso_vrp = PSO_VRP(random_state=100)
        results_pso_vrp = get_analysis(pso_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node_analysis["pso_vrp"].append(results_pso_vrp)
    
    doi_combi_analysis.append(random_node_analysis)

doi combination =  (0.15, 0.45, 0.77)
random state =  5
random state =  10
random state =  15
random state =  20
random state =  25
random state =  30
random state =  35
random state =  40
random state =  45
random state =  50
random state =  55
random state =  60
random state =  65
random state =  70
random state =  75
random state =  80
random state =  85
random state =  90
random state =  95
random state =  100
random state =  105
random state =  110
random state =  115
random state =  120
random state =  125
random state =  130
random state =  135
random state =  140
random state =  145
random state =  150
random state =  155
random state =  160
random state =  165
random state =  170
random state =  175
random state =  180
random state =  185
random state =  190
random state =  195
random state =  200
random state =  205
random state =  210
random state =  215
random state =  220
random state =  225
random state =  230
random state =  235
random state =  240
random state =  245
ra

random state =  10
random state =  15
random state =  20
random state =  25
random state =  30
random state =  35
random state =  40
random state =  45
random state =  50
random state =  55
random state =  60
random state =  65
random state =  70
random state =  75
random state =  80
random state =  85
random state =  90
random state =  95
random state =  100
random state =  105
random state =  110
random state =  115
random state =  120
random state =  125
random state =  130
random state =  135
random state =  140
random state =  145
random state =  150
random state =  155
random state =  160
random state =  165
random state =  170
random state =  175
random state =  180
random state =  185
random state =  190
random state =  195
random state =  200
random state =  205
random state =  210
random state =  215
random state =  220
random state =  225
random state =  230
random state =  235
random state =  240
random state =  245
random state =  250
doi combination =  (0.16, 0.65, 0.12)


In [33]:
doi_combi_analysis

[{'pso_vrp': [{'solution': [[88, 10, 7, 90, 4, 55, 52, 35, 50, 9],
     [51, 14, 2, 48, 74, 26],
     [34, 38, 99, 15, 22, 66, 83]],
    'fitness': 0.7798729131093666,
    'daily_fitness': [0.6381769188581942,
     0.5371743300102905,
     0.5000232200196377],
    'days_needed': 3,
    'poi_included': 23,
    'avg_rating': 4.578260869565218,
    'total_tarif': 92500,
    'total_duration': 126840,
    'run_time': 0.41261839866638184},
   {'solution': [[46, 66, 38, 51, 74, 57, 21],
     [44, 99, 18, 32, 22, 71, 49],
     [6, 10, 2, 12, 67, 48, 36, 5, 89]],
    'fitness': 0.7316498454843045,
    'daily_fitness': [0.5195048106244475,
     0.46411536578585993,
     0.48259073596953056],
    'days_needed': 3,
    'poi_included': 23,
    'avg_rating': 4.552173913043478,
    'total_tarif': 131500,
    'total_duration': 129360,
    'run_time': 0.803959846496582},
   {'solution': [[81, 32, 2, 20, 85, 64, 46],
     [68, 3, 22, 99, 15, 33, 43, 36],
     [8, 39, 50, 53, 5, 74, 57]],
    'fitness': 

In [34]:
result_to_xlsx_rd_doi(doi_combi_analysis,doi_combi,random_ls,"pso_random_doi_combi.xlsx")

## 87 POIs

In [35]:
def result_to_xlsx_all_poi(result,n,random_ls,filename):
    df_result = pd.DataFrame()
    for i in result:
        for j in range(len(result[i])):
            temp = pd.DataFrame(result[i][j],columns=["fitness","run_time"])
            temp['n'] = n[j]
            temp['method'] = i
            temp['random_seed'] = random_ls
            df_result = pd.concat([df_result,temp])
    df_result.to_excel(filename,index=False)

### All DOIs 1

In [36]:
n = [5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,87]
run_time_analysis = {
    "pso_vrp": []
}
max_runtime = 100*60 #100 minutes
for i in n:
    print("n = ",i)
    random_ls = list(range(5,(5*50)+1,5))
    random_node = {
        "pso_vrp": [],
    }
    
    for j in range(len(random_ls)):
        # setting
        hotel,tur,timematrix = generate_data(n=i,random_state=random_ls[j])
        travel_days = 3
        dwaktu,dtarif,drating = 1,1,1
        
        print("random_state : ",random_ls[j])
        
        pso_vrp = PSO_VRP(random_state=100)
        results_pso_vrp = get_analysis(pso_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        random_node["pso_vrp"].append(results_pso_vrp)
    
    for method in run_time_analysis:
        run_time_analysis[method].append(random_node[method])

n =  5
random_state :  5
random_state :  10
random_state :  15
random_state :  20
random_state :  25
random_state :  30
random_state :  35
random_state :  40
random_state :  45
random_state :  50
random_state :  55
random_state :  60
random_state :  65
random_state :  70
random_state :  75
random_state :  80
random_state :  85
random_state :  90
random_state :  95
random_state :  100
random_state :  105
random_state :  110
random_state :  115
random_state :  120
random_state :  125
random_state :  130
random_state :  135
random_state :  140
random_state :  145
random_state :  150
random_state :  155
random_state :  160
random_state :  165
random_state :  170
random_state :  175
random_state :  180
random_state :  185
random_state :  190
random_state :  195
random_state :  200
random_state :  205
random_state :  210
random_state :  215
random_state :  220
random_state :  225
random_state :  230
random_state :  235
random_state :  240
random_state :  245
random_state :  250
n =  10
rando

random_state :  80
random_state :  85
random_state :  90
random_state :  95
random_state :  100
random_state :  105
random_state :  110
random_state :  115
random_state :  120
random_state :  125
random_state :  130
random_state :  135
random_state :  140
random_state :  145
random_state :  150
random_state :  155
random_state :  160
random_state :  165
random_state :  170
random_state :  175
random_state :  180
random_state :  185
random_state :  190
random_state :  195
random_state :  200
random_state :  205
random_state :  210
random_state :  215
random_state :  220
random_state :  225
random_state :  230
random_state :  235
random_state :  240
random_state :  245
random_state :  250
n =  50
random_state :  5
random_state :  10
random_state :  15
random_state :  20
random_state :  25
random_state :  30
random_state :  35
random_state :  40
random_state :  45
random_state :  50
random_state :  55
random_state :  60
random_state :  65
random_state :  70
random_state :  75
random_state

random_state :  155
random_state :  160
random_state :  165
random_state :  170
random_state :  175
random_state :  180
random_state :  185
random_state :  190
random_state :  195
random_state :  200
random_state :  205
random_state :  210
random_state :  215
random_state :  220
random_state :  225
random_state :  230
random_state :  235
random_state :  240
random_state :  245
random_state :  250


In [37]:
run_time_analysis

{'pso_vrp': [[{'solution': [[93, 48, 88, 35, 99]],
    'fitness': 0.7043981481481482,
    'daily_fitness': [0.5565972222222222],
    'days_needed': 1,
    'poi_included': 5,
    'avg_rating': 4.58,
    'total_tarif': 52500,
    'total_duration': 28980,
    'run_time': 0.026240825653076172},
   {'solution': [[58, 66, 80, 5, 2]],
    'fitness': 0.7103086419753084,
    'daily_fitness': [0.5654629629629626],
    'days_needed': 1,
    'poi_included': 5,
    'avg_rating': 4.4799999999999995,
    'total_tarif': 70000,
    'total_duration': 38640,
    'run_time': 0.017690658569335938},
   {'solution': [[2, 71, 29, 22, 5]],
    'fitness': 0.7047839506172839,
    'daily_fitness': [0.5571759259259259],
    'days_needed': 1,
    'poi_included': 5,
    'avg_rating': 4.5,
    'total_tarif': 70000,
    'total_duration': 35160,
    'run_time': 0.015729427337646484},
   {'solution': [[21, 96, 36, 90, 97]],
    'fitness': 0.7191358024691361,
    'daily_fitness': [0.5787037037037043],
    'days_needed': 

In [38]:
result_to_xlsx_all_poi(run_time_analysis,n,random_ls,"pso_various_numbers_pois_all_1.xlsx")